In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import mixed_precision
from extra_models.object_detector import ObjectDetector
import matplotlib.pyplot as plt
from model_trainer import create_train_dataset

In [ ]:
%matplotlib inline

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

In [ ]:
data_dir = 'data/val'
img_size = (640,480)
class_names = ['food']
bbox_sizes = [(25,25)]

In [ ]:
ds = create_train_dataset(data_dir, img_size, class_names, bbox_sizes, val_data=True)

In [ ]:
backbone_f = 'hr_5_3_8'
intermediate_filters = 256
kernel_size = 16
stride = 8
num_classes = len(class_names) + 1
rfcn_window=3
anchor_ratios = [0.5,1.0,2.0]
anchor_scales = [0.1,0.3,0.6]


In [ ]:
test_model = ObjectDetector(
    backbone_f,
    intermediate_filters,
    kernel_size,
    stride,
    img_size,
    num_classes,
    rfcn_window,
    anchor_ratios,
    anchor_scales,
)

In [ ]:
load_dir = 'savedmodels/hr538_rfcn_mf8/15'
test_model.load_weights(load_dir,)

In [ ]:
colors=np.array([
    [1,0,0],
    [0,1,0],
    [1,1,0],
    [0,1,1],
])


samples = ds.take(4).as_numpy_iterator()
fig = plt.figure(figsize=(15,15))
for i in range(4):
    sample = next(samples)
    image, gt_box, _ = sample
    #-------------DEBUG
    rois, rpn_probs, boxes, probs, labels= test_model(image, training=False)
    rpn_image = image[0].copy()
    #---------------------------------------
    # boxes, probs, labels = test_model(image, training=False)
    #----------------------------------------
    test_image = image[0].copy()
    gt_image = image[0].copy()
    gt_box = gt_box[0]
    h,w = np.subtract(gt_image.shape[:2],1)
    #----------------------DEBUG
    for box, p in zip(rois,rpn_probs):
        color = np.clip([4*(1-p),4*(p-0.5),0],0,1)
        x1, y1, x2, y2 = np.multiply(box,[w,h,w,h,]).astype(np.int64)
        rpn_image[y1,x1:x2] = color
        rpn_image[y2,x1:x2] = color
        rpn_image[y1:y2,x1] = color
        rpn_image[y1:y2,x2] = color
    #----------------------------------
    for box, p, l in zip(boxes, probs, labels):
        color = colors[l] * p
    #----------------------------------------
        x1, y1, x2, y2 = np.multiply(box,[w,h,w,h,]).astype(np.int64)
        test_image[y1,x1:x2] = color
        test_image[y2,x1:x2] = color
        test_image[y1:y2,x1] = color
        test_image[y1:y2,x2] = color
    for box in gt_box:
        x1, y1, x2, y2 = np.multiply(box,[w,h,w,h,]).astype(np.int64)
        gt_image[y1,x1:x2] = [0,0,1]
        gt_image[y2,x1:x2] = [0,0,1]
        gt_image[y1:y2,x1] = [0,0,1]
        gt_image[y1:y2,x2] = [0,0,1]

    ax = fig.add_subplot(4,3,3*i+1)
    ax.imshow(gt_image)
    ax = fig.add_subplot(4,3,3*i+2)
    ax.imshow(rpn_image)
    ax = fig.add_subplot(4,3,3*i+3)
    ax.imshow(test_image)

plt.show()
